## 간단한 구조의 CNN 모델 살펴보기

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam

# 각 층이 가지는 여러 가지 하이퍼파라미터에 집중합시다.
inputs = Input(shape = (28, 28, 1))

x = Conv2D(32, (3, 3), activation = 'relu')(inputs)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = MaxPooling2D(strides = 2)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(30, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = x)

model.compile(optimizer = Adam(learning_rate = 0.001), 
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

## 케라스 튜너 모델 정의하기

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    inputs = Input(shape = (28, 28, 1))
    x = inputs

    for i in range(hp.Int('n_layers', 1, 3)):
    # 필터 개수를 조정합니다.
    x = Conv2D(
        filters = hp.Int('filters_' + str(i), 4, 64, step = 8, default = 16),
        kernel_size = (3, 3), activation = 'relu',
        padding = 'same')(x)
    x = MaxPooling2D(strides = 2)(x)

    # GAP? GMP?
    if hp.Choice('global_pooling', ['max', 'avg']) == 'avg':
        x = GlobalAveragePooling2D()(x)
    else:
        x = GlobalMaxPooling2D()(x)

    # 은닉층의 개수를 조정합니다.
    x = Dense(units = hp.Int('units', 
                          min_value = 16,
                          max_value = 128,
                          step = 16),
            activation = 'relu')(x)
    # 드롭아웃률을 조정합니다.
    x = Dropout(hp.Choice('dropout_rate', values = [0.2, 0.3, 0.5]))(x)
    x = Dense(10, activation = 'softmax')(x)

    model = Model(inputs = inputs, outputs = x)
    model.compile(optimizer = Adam(hp.Choice('learning_rate', values = [1e-3, 1e-4, 1e-5])),
                loss = 'categorical_crossentropy',
                metrics = ['acc'])

    return model

## MNIST 데이터셋 준비하기

In [ ]:
from tensorflow.keras.datasets import mnist

# 텐서플로우 저장소에서 데이터를 다운로드 받습니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data(path = 'mnist.npz')

from sklearn.model_selection import train_test_split

# 훈련/검증 데이터를 얻기 위해 0.7/0.3의 비율로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, random_state = 777)

num_x_train = x_train.shape[0]
num_x_val = x_val.shape[0]
num_x_test = x_test.shape[0]

# 모델의 입력으로 사용하기 위한 전처리 과정입니다.
x_train = (x_train.reshape(-1, 28, 28, 1)) / 255
x_val = (x_val.reshape(-1, 28, 28, 1)) / 255
x_test = (x_test.reshape(-1, 28, 28, 1)) / 255

from tensorflow.keras.utils import to_categorical

# 각 데이터의 레이블을 범주형 형태로 변경합니다.
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

## RandomSearch 클래스 사용하기

In [ ]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(build_model,
                     objective='val_acc',
                     max_trials=5,
                     executions_per_trial=3,
                     directory='mypath', # 본인의 경로로 수정하세요.
                     project_name='helloworld')

## 탐색할 하이퍼파라미터 살펴보기

In [ ]:
tuner.search_space_summary()

## 하이퍼파라미터 탐색하기

In [ ]:
tuner.search(x=x_train,y=y_train,
             epochs=10,
             validation_data=(x_val, y_val))

## 실험 결과 요약해보기

In [ ]:
tuner.results_summary()

## 가장 좋은 성능의 모델 불러오기

In [ ]:
best_model = tuner.get_best_models()[0]
best_model.summary()

## 모델 하이퍼파라미터 확인하기

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0].values
best_hp

## HyperResNet 사용하기

In [ ]:
from kerastuner.tuners import Hyperband
from kerastuner.applications import HyperResNet

hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

tuner = Hyperband(hypermodel,
                  objective = 'val_accuracy',
                  max_epochs = 10,
                  directory='./drive/My Drive/tf2.0intro/code/ch9/save_tuner/',
                  project_name='helloworld2')

In [ ]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_val, y_val))